In [1]:
from lxml.html import parse
from urllib.request import urlopen
from pandas.io.parsers import TextParser

In [2]:
parsed = parse(urlopen('https://www.sec.gov/Archives/edgar/data/51143/000005114313000007/0000051143-13-000007-index.htm'))
doc = parsed.getroot()

In [3]:
links = doc.findall('.//a')

In [4]:
url = [lnk.get('href')for lnk in doc.findall('.//a')]

In [5]:
url[10:23]

['/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex11.htm',
 '/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex12.htm',
 '/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex31d1.htm',
 '/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex31d2.htm',
 '/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex32d1.htm',
 '/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex32d2.htm',
 '/Archives/edgar/data/51143/000005114313000007/0000051143-13-000007.txt',
 '/Archives/edgar/data/51143/000005114313000007/ibm-20130930.xml',
 '/Archives/edgar/data/51143/000005114313000007/ibm-20130930.xsd',
 '/Archives/edgar/data/51143/000005114313000007/ibm-20130930_cal.xml',
 '/Archives/edgar/data/51143/000005114313000007/ibm-20130930_def.xml',
 '/Archives/edgar/data/51143/000005114313000007/ibm-20130930_lab.xml',
 '/Archives/edgar/data/51143/000005114313000007/ibm-20130930_pre.xml']

In [16]:
tables = doc.findall('.//table')

In [17]:
#len(tables)

2

In [7]:
#for x in len(tables):
example = tables[0]

In [8]:
rows = example.findall('.//tr')

In [9]:
def _unpack(row, kind='td'):
    elts = row.findall('.//%s' % kind)
    return [val.text_content() for val in elts]

In [10]:
print(_unpack(rows[0], kind="th"))

['Seq', 'Description', 'Document', 'Type', 'Size']


In [11]:
def parse_options_data(table):
    rows=table.findall('.//tr')
    header = _unpack(rows[0], kind ='th')
    data = [_unpack(r) for r in rows[1:]]
    return TextParser(data, names = header).get_chunk()

In [18]:
example_data = parse_options_data(example)
example_data[1:10]

,Seq,Description,Document,Type,Size
1,2,EX-11,ibm13q3_ex11.htm,EX-11,81348
2,3,EX-12,ibm13q3_ex12.htm,EX-12,35495
3,4,EX-31.1,ibm13q3_ex31d1.htm,EX-31.1,10428
4,5,EX-31.2,ibm13q3_ex31d2.htm,EX-31.2,11028
5,6,EX-32.1,ibm13q3_ex32d1.htm,EX-32.1,6773
6,7,EX-32.2,ibm13q3_ex32d2.htm,EX-32.2,7623
7,,Complete submission text file,0000051143-13-000007.txt,,26894261


In [13]:
for decription in example_data['Description']:
    if decription == '10q'
example_data['Document']

0             ibm13q3_10q.htm
1            ibm13q3_ex11.htm
2            ibm13q3_ex12.htm
3          ibm13q3_ex31d1.htm
4          ibm13q3_ex31d2.htm
5          ibm13q3_ex32d1.htm
6          ibm13q3_ex32d2.htm
7    0000051143-13-000007.txt
Name: Document, dtype: object